In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("/nfs/turbo/coe-chaijy/sstorks/simulation_informed_pcr4nlu/TRAVEl")
from travel import init_travel
init_travel()

In [3]:
from transformers import AutoModelForCausalLM

lm = AutoModelForCausalLM.from_pretrained("gpt2")

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [9]:
from pprint import pprint

prompt = "Hello world"
tokens = tokenizer(prompt, return_tensors="pt")
outputs = lm.generate(**tokens)
pprint(outputs)
outputs_decoded = tokenizer.batch_decode(outputs)

pprint(outputs_decoded)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


tensor([[15496,   995,    11,   314,  1101,   407,  1654,   644,   284,   910,
            13,   198,   198,     1,    40,  1101,  7926,    11,   475,   314]])
['Hello world, I\'m not sure what to say.\n\n"I\'m sorry, but I']


# Load preds

In [2]:
import json
from pprint import pprint

RESULTS_PATH_LIKELIHOOD = "/home/sstorks/coe-chaijy/sstorks/simulation_informed_pcr4nlu/TRAVEl/saved_results_222/vqa_mistake_detection/ego4d_single_debug250/llava-1.5-7b-hf/IterativeVQA_q10_ego4d_single_debug250_llava-1.5-7b-hf_beam8-4_likelihood_nohistory_20240815204213"
preds_l = json.load(open(os.path.join(RESULTS_PATH_LIKELIHOOD, "outputs_val.json"), "r"))
metrics_l = json.load(open(os.path.join(RESULTS_PATH_LIKELIHOOD, "metrics_coherence_nli_val.json"), "r"))
combined_l = [v | {"example_id": k, 
                   "relevance_marginal": metrics_l['metrics_by_turn']['relevance_marginal_by_turn'][i], 
                   "informativeness": metrics_l['metrics_by_turn']['informativeness_by_turn'][i], 
                   "informativeness_marginal": metrics_l['metrics_by_turn']['informativeness_marginal_by_turn'][i],
                   "informativeness_marginal_x_relevance_marginal": metrics_l['metrics_by_turn']['informativeness_marginal_x_relevance_marginal_by_turn'][i],
                   "informativeness_marginal_x_relevance_marginal_ref": metrics_l['metrics_by_turn']['informativeness_marginal_x_relevance_marginal_ref_by_turn'][i]
                  } for i, (k, v) in enumerate(preds_l.items())]

combined_c = []
RESULTS_PATH_COHERENCE = "/home/sstorks/coe-chaijy/sstorks/simulation_informed_pcr4nlu/TRAVEl/saved_results_222/vqa_mistake_detection/ego4d_single_debug250/llava-1.5-7b-hf/IterativeVQA_q10_ego4d_single_debug250_llava-1.5-7b-hf_beam8-4_coherence_icl20_nohistory_20240815204213"
preds_c = json.load(open(os.path.join(RESULTS_PATH_COHERENCE, "outputs_val.json"), "r"))
metrics_c = json.load(open(os.path.join(RESULTS_PATH_COHERENCE, "metrics_coherence_nli_val.json"), "r"))
combined_c = [v | {"example_id": k, 
                   "informativeness": metrics_c['metrics_by_turn']['informativeness_by_turn'][i], 
                   "relevance_marginal": metrics_c['metrics_by_turn']['relevance_marginal_by_turn'][i], 
                   "informativeness_marginal": metrics_c['metrics_by_turn']['informativeness_marginal_by_turn'][i],
                   "informativeness_marginal_x_relevance_marginal": metrics_c['metrics_by_turn']['informativeness_marginal_x_relevance_marginal_by_turn'][i],
                   "informativeness_marginal_x_relevance_marginal_ref": metrics_c['metrics_by_turn']['informativeness_marginal_x_relevance_marginal_ref_by_turn'][i],
                  } for i, (k, v) in enumerate(preds_c.items())]

all_data = combined_l + combined_c

pprint(all_data[0])

{'answer_probs': [[0.010489, 0.989511],
                  [0.7773, 0.2227],
                  [0.007233, 0.992767],
                  [0.651355, 0.348645],
                  [0.307358, 0.692642],
                  [0.33112, 0.66888],
                  [0.573678, 0.426322],
                  [0.73412, 0.26588],
                  [0.538983, 0.461017],
                  [0.72487, 0.27513]],
 'answers': ['Yes',
             'No',
             'Yes',
             'No',
             'Yes',
             'Yes',
             'Unsure',
             'No',
             'Unsure',
             'No'],
 'candidate_questions': [['Is the person wearing a lab coat?',
                          'Is the person wearing gloves?',
                          'Is the person wearing a white lab coat?'],
                         ['Is the person wearing safety goggles?',
                          'Is the person wearing gloves?'],
                         ['Is the person wearing gloves?',
                          'I

# Select turns and candidates to score

Relevance study:

In [3]:
import datetime
import numpy as np
import os
import random

from travel import set_random_seed
from travel.constants import RANDOM_SEED, RESULTS_DIR

N_SAMPLES = 100
SAMPLES_PER_ANNOTATOR = 10

timestamp = datetime.datetime.now()
output_dir = os.path.join("annotation_files", f"relevance_{timestamp.strftime('%Y%m%d%H%M%S')}")
os.makedirs(output_dir)

random_seed = 555
set_random_seed(random_seed) # Set seed to 555 for relevance samples

example_idxs = random.sample(list(range(len(all_data))), N_SAMPLES)

data_dicts = []
for example_idx in example_idxs:   
    output = all_data[example_idx]
        
    example_id = output['example_id']

    n_turns = output['final_turn'] + 1
    selected_turn = random.choice(list(range(n_turns)))

    selected_question = output['questions'][selected_turn]
    selected_answer = np.argmax(output['answer_probs'][selected_turn])
    assert selected_answer == 0 or selected_answer == 1
    selected_answer = "Yes" if selected_answer == 1 else "No"

    previous_questions = [q for q, a in zip(output['questions'][:selected_turn], output['answers'][:selected_turn]) if a != "Unsure"]
    previous_answers = [a for a in output['answers'][:selected_turn] if a != "Unsure"]

    actual_relevance_marginal = output['relevance_marginal'][selected_turn]
    actual_informativeness = output['informativeness'][selected_turn]
    actual_informativeness_marginal = output['informativeness_marginal'][selected_turn]
    max_informativeness_marginal_so_far = max([turn_metric for turn_idx, turn_metric in enumerate(output['informativeness_marginal'][:selected_turn+1]) if output['answers'][turn_idx] != "Unsure" or turn_idx == selected_turn])
    max_informativeness_marginal_so_far_x_mean_relevance_marginal = round(max_informativeness_marginal_so_far * np.mean(output['relevance_marginal'][:selected_turn + 1]), 6)
    actual_informativeness_marginal_x_relevance_marginal = output['informativeness_marginal_x_relevance_marginal'][selected_turn]
    actual_informativeness_marginal_x_relevance_marginal_ref = output['informativeness_marginal_x_relevance_marginal_ref'][selected_turn]
                                  
    data_dicts.append(
        {
            "example_id": example_id,
            "procedure": output['procedure'],
            "question": selected_question,
            "answer": selected_answer,
            "previous_questions_answers": [(q, a) for q, a in zip(previous_questions, previous_answers)],
            "nli_metric_scores": {
                "relevance_marginal": actual_relevance_marginal,
                "informativeness": actual_informativeness,
                "informativeness_x_relevance_marginal": actual_informativeness * actual_relevance_marginal, # TODO: later can just get this directly from metrics
                "informativeness_marginal": actual_informativeness_marginal,
                "max_informativeness_marginal_so_far": max_informativeness_marginal_so_far,
                "max_informativeness_marginal_so_far_x_mean_relevance_marginal": max_informativeness_marginal_so_far_x_mean_relevance_marginal,
                "informativeness_marginal_x_relevance_marginal": actual_informativeness_marginal_x_relevance_marginal,
                "informativeness_marginal_x_relevance_marginal_ref": actual_informativeness_marginal_x_relevance_marginal_ref,
            }
        }
    )

json.dump(data_dicts, open(os.path.join(output_dir, f"relevance_sample{random_seed}.json"), "w"), indent=4)

In [4]:
# Divide sampled data into a txt file for each annotator
for annotator_idx in range(N_SAMPLES // SAMPLES_PER_ANNOTATOR):
    lines = []

    samples = data_dicts[annotator_idx * SAMPLES_PER_ANNOTATOR:(annotator_idx + 1) * SAMPLES_PER_ANNOTATOR]
    assert len(samples) == SAMPLES_PER_ANNOTATOR
    
    lines.append("Imagine you just had eye surgery, and are unable to see. You're performing a task you're familiar with, but need help to determine whether you successfully completed it.")
    lines.append("You video call a friend (who is unfamiliar with the task) and show them what you're working on. You then ask them some yes/no questions to figure out whether you successfully completed the task.")
    lines.append("")
    lines.append("For each annotation task, you will be given the following information:")
    lines.append(" - A sentence describing the procedure you're trying to perform.")
    lines.append(" - An optional list of previous questions you already asked, and their answers.")
    lines.append(" - A potential question you could ask your friend next.")
    lines.append("")
    lines.append("You must rate how relevant the potential next question is. By relevant, we mean: given the previous questions and answers, how helpful could an answer to this question be in determining whether the procedure has been completed?")
    lines.append("You can also choose to mark 'Instructions Unclear', which means that the instructional text itself is not clear, so you're not sure how to determine whether it's successful. This should only be used in rare cases.")
    lines.append("")
    lines.append("Some tips:")
    lines.append(" - Only judge the relevance of the potential next question, not the previous questions (which may or may not be relevant).")
    lines.append(" - A question may seem relevant to the task at hand, but you should consider it irrelevant if it doesn't provide essential information to judge the success of the task.")
    lines.append(" - If a seemingly relevant question is redundant with previous questions, you may consider it less relevant.")
    lines.append(" - Assume that the answer to the question won't contradict the information you have from previous questions and answers. If previous questions and answers already contradict each other, consider whether this question could sway you one way or another.")
    lines.append(' - The instructional text and questions may refer to "someone" or a "person"; always assume this is referring to yourself (the person performing the task).')
    lines.append(' - The questions may refer to a "photo" or "image"; always assume this is referring to the video feed your friend would see through the video call.')
    lines.append("\n")
                 
    # How relevant is this question, i.e., how useful would an answer to this question be in determining whether the procedure has been completed?
    for sample_idx, sample in enumerate(samples):
        lines.append(f"Annotation {sample_idx + 1}")
        lines.append("===============================")
        lines.append(f"Instruction: {sample['procedure']}")
        
        lines.append("Previous questions and answers:")
        if len(sample['previous_questions_answers']) == 0:
            lines.append("None")
        else:
            for q_idx, (q, a) in enumerate(sample['previous_questions_answers']):
                 lines.append(f"Q{q_idx+1}. {q}     (Answer: {a})")
        lines.append("")
        lines.append(f"Potential next question: {sample['question']}")
        lines.append("Your rating (place an 'x' next to your choice):")
        lines.append("1 (very irrelevant): ")
        lines.append("2 (slightly irrelevant): ")
        lines.append("3 (neutral): ")
        lines.append("4 (slightly relevant): ")
        lines.append("5 (very relevant): ")
        lines.append("Instructions Unclear: ")
        lines.append("\n")
                 
    with open(os.path.join(output_dir, f"relevance_sample{random_seed}_annotator{annotator_idx+1}.txt"), "w") as f:
        f.write("\n".join(lines))

Informativeness study:

In [4]:
import datetime
import os
import random
import numpy as np

from travel import set_random_seed
from travel.constants import RANDOM_SEED, RESULTS_DIR

N_SAMPLES = 100
SAMPLES_PER_ANNOTATOR = 10

timestamp = datetime.datetime.now()
output_dir = os.path.join("annotation_files", f"informativeness_{timestamp.strftime('%Y%m%d%H%M%S')}")
os.makedirs(output_dir)

random_seed = 888
set_random_seed(random_seed) # Set seed to 888 for informativeness samples

example_idxs = random.sample(list(range(len(all_data))), N_SAMPLES)

data_dicts = []
for example_idx in example_idxs:
    output = all_data[example_idx]
    example_id = output['example_id']

    n_turns = output['final_turn'] + 1
    selected_turn = random.choice(list(range(n_turns)))

    selected_question = output['questions'][selected_turn]
    selected_answer = np.argmax(output['answer_probs'][selected_turn])
    assert selected_answer == 0 or selected_answer == 1
    selected_answer = "Yes" if selected_answer == 1 else "No"

    previous_questions = [q for q, a in zip(output['questions'][:selected_turn], output['answers'][:selected_turn]) if a != "Unsure"]
    previous_answers = [a for a in output['answers'][:selected_turn] if a != "Unsure"]

    actual_relevance_marginal = output['relevance_marginal'][selected_turn]
    actual_informativeness = output['informativeness'][selected_turn]
    actual_informativeness_marginal = output['informativeness_marginal'][selected_turn]
    max_informativeness_marginal_so_far = max([turn_metric for turn_idx, turn_metric in enumerate(output['informativeness_marginal'][:selected_turn+1]) if output['answers'][turn_idx] != "Unsure" or turn_idx == selected_turn])
    max_informativeness_marginal_so_far_x_mean_relevance_marginal = round(max_informativeness_marginal_so_far * np.mean(output['relevance_marginal'][:selected_turn + 1]), 6)
    actual_informativeness_marginal_x_relevance_marginal = output['informativeness_marginal_x_relevance_marginal'][selected_turn]
    actual_informativeness_marginal_x_relevance_marginal_ref = output['informativeness_marginal_x_relevance_marginal_ref'][selected_turn]
    
    data_dicts.append(
        {
            "example_id": example_id,
            "procedure": output['procedure'],
            "question": selected_question,
            "answer": selected_answer,
            "previous_questions_answers": [(q, a) for q, a in zip(previous_questions, previous_answers)],
            "nli_metric_scores": {
                "relevance_marginal": actual_relevance_marginal,
                "informativeness": actual_informativeness,
                "informativeness_x_relevance_marginal": actual_informativeness * actual_relevance_marginal, # TODO: later can just get this directly from metrics
                "informativeness_marginal": actual_informativeness_marginal,
                "max_informativeness_marginal_so_far": max_informativeness_marginal_so_far,
                "max_informativeness_marginal_so_far_x_mean_relevance_marginal": max_informativeness_marginal_so_far_x_mean_relevance_marginal,
                "informativeness_marginal_x_relevance_marginal": actual_informativeness_marginal_x_relevance_marginal,
                "informativeness_marginal_x_relevance_marginal_ref": actual_informativeness_marginal_x_relevance_marginal_ref,
            }
        }
    )

json.dump(data_dicts, open(os.path.join(output_dir, f"informativeness_sample{random_seed}.json"), "w"), indent=4)

In [5]:
# Divide sampled data into a txt file for each annotator
for annotator_idx in range(N_SAMPLES // SAMPLES_PER_ANNOTATOR):
    lines = []

    samples = data_dicts[annotator_idx * SAMPLES_PER_ANNOTATOR:(annotator_idx + 1) * SAMPLES_PER_ANNOTATOR]
    assert len(samples) == SAMPLES_PER_ANNOTATOR
    
    lines.append("Imagine you just had eye surgery, and are unable to see. You're performing a task you're familiar with, but need help to determine whether you successfully completed it.")
    lines.append("You video call a friend (who is unfamiliar with the task) and show them what you're working on. You then ask them some yes/no questions to figure out whether you successfully completed the task.")
    lines.append("")
    lines.append("For each annotation task, you will be given the following information:")
    lines.append(" - A sentence describing the procedure you're trying to perform.")
    lines.append(" - An optional list of previous questions you already asked, and their answers.")
    lines.append(" - The last question you just asked your friend, and its answer.")
    lines.append("")
    lines.append("You must rate how informative the last question and its answer are. By informative, we mean: compared to what you knew from the previous questions and answers, how much more sure would the last question and answer be about whether you succeeded?")
    lines.append("You can also choose to mark 'Instructions Unclear', which means that the instructional text itself is not clear, so you're not sure how to determine whether it's successful. This should only be used in rare cases.")
    lines.append("")
    lines.append("Some tips:")
    lines.append(" - Only judge the informativeness of the last question and answer, not the previous questions and answers (which may or may not be informative).")
    lines.append(" - A question may seem relevant to the task at hand, but you should consider it uninformative if it doesn't provide essential information to judge the success of the task.")
    lines.append(" - If a seemingly informative question is redundant with previous questions, you may consider it less informative.")
    lines.append(" - If the last answer contradicts critical information you had from previous questions and answers, you may consider it more informative.")
    lines.append(' - The instructional text and questions may refer to "someone" or a "person"; always assume this is referring to yourself (the person performing the task).')
    lines.append(' - The questions may refer to a "photo" or "image"; always assume this is referring to the video feed your friend would see through the video call.')
    lines.append("\n")
                 
    # How relevant is this question, i.e., how useful would an answer to this question be in determining whether the procedure has been completed?
    for sample_idx, sample in enumerate(samples):
        lines.append(f"Annotation {sample_idx + 1}")
        lines.append("===============================")
        lines.append(f"Instruction: {sample['procedure']}")
        
        lines.append("Previous questions and answers:")
        if len(sample['previous_questions_answers']) == 0:
            lines.append("None")
        else:
            for q_idx, (q, a) in enumerate(sample['previous_questions_answers']):
                 lines.append(f"Q{q_idx+1}. {q}     (Answer: {a})")
        lines.append("")
        lines.append(f"Last question: {sample['question']}")
        lines.append(f"Last answer: {sample['answer']}")
        lines.append("Your rating (place an 'x' next to your choice):")
        lines.append("1 (very informative): ")
        lines.append("2 (slightly informative): ")
        lines.append("3 (neutral): ")
        lines.append("4 (slightly informative): ")
        lines.append("5 (very informative): ")
        lines.append("Instructions Unclear: ")
        lines.append("\n")
                 
    with open(os.path.join(output_dir, f"informativeness_sample{random_seed}_annotator{annotator_idx+1}.txt"), "w") as f:
        f.write("\n".join(lines))

# Checking correlations of annotations with metrics

In [16]:
import json
import os
import pandas as pd
from pprint import pprint

# INPUT_FILE = "annotation_files/relevance_20240816145229/relevance_sample555.json"
# ANNOTATIONS_DIR = "../informativeness_relevance_annotation/relevance"
# metric_key = 'relevance_marginal'

INPUT_FILE = "annotation_files/informativeness_20240817125001/informativeness_sample888.json"
ANNOTATIONS_DIR = "../informativeness_relevance_annotation/informativeness"
metric_key = "informativeness_marginal_x_relevance_marginal"
# metric_key = "informativeness_marginal"

data_dicts = json.load(open(INPUT_FILE, "r"))

all_annotations = None
for fname in os.listdir(ANNOTATIONS_DIR):
    d = pd.read_csv(os.path.join(ANNOTATIONS_DIR, fname))
    
    if all_annotations is None:
        all_annotations = d
    else:
        all_annotations = pd.concat((all_annotations, d))

(optional) recalculate metrics

In [17]:
import torch
from transformers import AutoModelForVision2Seq, AutoProcessor, BitsAndBytesConfig, AutoModelForCausalLM

# VLM_NAME = "Salesforce/instructblip-vicuna-7b"
# VLM_NAME = "Salesforce/instructblip-flan-t5-xxl"
VLM_NAME = "llava-hf/llava-1.5-7b-hf"
# VLM_NAME = "microsoft/Phi-3-vision-128k-instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

try:
    vlm = AutoModelForVision2Seq.from_pretrained(VLM_NAME, 
                                                quantization_config=bnb_config, trust_remote_code=True)
except:
    vlm = AutoModelForCausalLM.from_pretrained(VLM_NAME, 
                                                quantization_config=bnb_config, trust_remote_code=True)
    
vlm_processor = AutoProcessor.from_pretrained(VLM_NAME, trust_remote_code=True)
vlm_processor.tokenizer.padding_side = "left"
vlm_processor.tokenizer.pad_token_id = vlm_processor.tokenizer.eos_token_id

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some kwargs in processor config are unused and will not have any effect: num_additional_image_tokens. 


In [18]:
import torch
from transformers import BitsAndBytesConfig, AutoModelForSequenceClassification, AutoTokenizer
import spacy

from travel.model.nli import NLI_MODEL_PATH
from travel.model.metrics import question_coherence_metrics_nli

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

nli_model = AutoModelForSequenceClassification.from_pretrained(NLI_MODEL_PATH, quantization_config=bnb_config)
nli_tokenizer = AutoTokenizer.from_pretrained(NLI_MODEL_PATH)
nlp = spacy.load("en_core_web_lg")


`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [19]:
from tqdm import tqdm

for annotation_idx in tqdm(range(len(data_dicts))):
    procedure = data_dicts[annotation_idx]['procedure']
    question = data_dicts[annotation_idx]['question']
    answer = data_dicts[annotation_idx]['answer']
    previous_questions = [q for q, _ in data_dicts[annotation_idx]['previous_questions_answers']]
    previous_answers = [a for _, a in data_dicts[annotation_idx]['previous_questions_answers']]
    
    updated_metrics = question_coherence_metrics_nli(nli_tokenizer, 
                                                     nli_model, 
                                                     vlm_processor.tokenizer,
                                                     vlm.language_model, 
                                                     [procedure],
                                                     [question],
                                                     [answer],
                                                     [previous_questions],
                                                     [previous_answers],
                                                     rephrase_success=False)
     
    print(f"Old value ({metric_key}):", data_dicts[annotation_idx]['nli_metric_scores'][metric_key])
    data_dicts[annotation_idx]['nli_metric_scores'][metric_key] = updated_metrics[metric_key]
    print(f"New value ({metric_key}):", data_dicts[annotation_idx]['nli_metric_scores'][metric_key])

print("Done!")    

running generation (cuda:0): 100%|██████████| 1/1 [00:01<00:00,  1.22s/it]

running generation (cuda:0): 100%|██████████| 1/1 [00:01<00:00,  1.21s/it]

running NLI (cuda:0): 100%|██████████| 1/1 [00:00<00:00, 15.47it/s]

running NLI (cuda:0): 100%|██████████| 1/1 [00:00<00:00, 16.91it/s]

running generation (cuda:0): 100%|██████████| 1/1 [00:02<00:00,  2.39s/it]

running NLI (cuda:0): 100%|██████████| 1/1 [00:00<00:00, 17.42it/s]

  1%|          | 1/100 [00:05<08:22,  5.08s/it]

Old value (informativeness_marginal_x_relevance_marginal): 0.011093
New value (informativeness_marginal_x_relevance_marginal): [0.020676]



running generation (cuda:0): 100%|██████████| 1/1 [00:01<00:00,  1.22s/it]

running generation (cuda:0): 100%|██████████| 1/1 [00:01<00:00,  1.23s/it]

running NLI (cuda:0): 100%|██████████| 1/1 [00:00<00:00, 17.14it/s]

running NLI (cuda:0): 100%|██████████| 1/1 [00:00<00:00, 17.24it/s]

running generation (cuda:0): 100%|██████████| 1/1 [00:02<00:00,  2.44s/it]

running NLI (cuda:0): 100%|██████████| 1/1 [00:00<00:00, 16.43it/s]

  2%|▏         | 2/100 [00:10<08:21,  5.12s/it]

Old value (informativeness_marginal_x_relevance_marginal): 0.025803
New value (informativeness_marginal_x_relevance_marginal): [0.021973]



running generation (cuda:0): 100%|██████████| 1/1 [00:01<00:00,  1.22s/it]

running generation (cuda:0): 100%|██████████| 1/1 [00:01<00:00,  1.21s/it]

running NLI (cuda:0): 100%|██████████| 1/1 [00:00<00:00, 17.00it/s]

running NLI (cuda:0): 100%|██████████| 1/1 [00:00<00:00, 17.29it/s]

running generation (cuda:0): 0it [00:00, ?it/s]

running NLI (cuda:0): 100%|██████████| 1/1 [00:00<00:00, 17.23it/s]

  3%|▎         | 3/100 [00:12<06:28,  4.00s/it]

...

Old value (informativeness_marginal_x_relevance_marginal): 0.02478
New value (informativeness_marginal_x_relevance_marginal): [0.020706]



running generation (cuda:0): 100%|██████████| 1/1 [00:01<00:00,  1.21s/it]

running generation (cuda:0): 100%|██████████| 1/1 [00:01<00:00,  1.21s/it]

running NLI (cuda:0): 100%|██████████| 1/1 [00:00<00:00, 16.73it/s]

running NLI (cuda:0): 100%|██████████| 1/1 [00:00<00:00, 17.04it/s]

running generation (cuda:0): 100%|██████████| 1/1 [00:02<00:00,  2.37s/it]

running NLI (cuda:0): 100%|██████████| 1/1 [00:00<00:00, 17.39it/s]

 99%|█████████▉| 99/100 [05:48<00:03,  3.87s/it]

Old value (informativeness_marginal_x_relevance_marginal): 0.000465
New value (informativeness_marginal_x_relevance_marginal): [3.1e-05]



running generation (cuda:0): 100%|██████████| 1/1 [00:01<00:00,  1.21s/it]

running generation (cuda:0): 100%|██████████| 1/1 [00:01<00:00,  1.20s/it]

running NLI (cuda:0): 100%|██████████| 1/1 [00:00<00:00, 17.46it/s]

running NLI (cuda:0): 100%|██████████| 1/1 [00:00<00:00, 17.56it/s]

running generation (cuda:0): 100%|██████████| 1/1 [00:02<00:00,  2.46s/it]

running NLI (cuda:0): 100%|██████████| 1/1 [00:00<00:00, 17.33it/s]

100%|██████████| 100/100 [05:53<00:00,  3.53s/it]

Old value (informativeness_marginal_x_relevance_marginal): 0.727539
New value (informativeness_marginal_x_relevance_marginal): [0.670898]
Done!


In [20]:
from pprint import pprint

N_SAMPLES = 100
SAMPLES_PER_ANNOTATOR = 10

calculated_values = []
annotated_values = []
for annotator_idx in range(N_SAMPLES // SAMPLES_PER_ANNOTATOR):
    samples = data_dicts[annotator_idx * SAMPLES_PER_ANNOTATOR:(annotator_idx + 1) * SAMPLES_PER_ANNOTATOR]
    assert len(samples) == SAMPLES_PER_ANNOTATOR
    
    for annotation_idx in range(len(samples)):
    
        retrieved_annotations = all_annotations[all_annotations['annotator_index'] == annotator_idx]
        retrieved_annotations = retrieved_annotations[retrieved_annotations['annotation_index'] == annotation_idx]
        if len(retrieved_annotations) == 0:
            print(f"Warning: did not find annotations for annotator {annotator_idx}, annotation {annotation_idx}. This may be expected if we did not have annotators do all the annotations in the source file.")
            continue
            
        assert len(retrieved_annotations) == 1, "Do we have some duplicate annotations?"
        
        rating = retrieved_annotations['rating'].iloc[0]
        if "Instructions Unclear" in rating:
            print("Warning: threw away an instance due to instructions being labeled unclear")
            continue
            
        rating = int(rating.split()[0])
        actual = samples[annotation_idx]['nli_metric_scores'][metric_key]
        
        calculated_values.append(actual)
        annotated_values.append(rating)
        
print(f"Recovered {len(annotated_values)} annotations.")
    

Recovered 50 annotations.


Correlation result:

In [21]:
from scipy.stats import spearmanr, pearsonr

res = spearmanr(calculated_values, annotated_values)
pprint(res)

res = pearsonr(calculated_values, annotated_values)
pprint(res)

SignificanceResult(statistic=0.4999466247399133, pvalue=0.000218405481196467)
PearsonRResult(statistic=array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]), pvalue=array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]))


/tmp/ipykernel_45180/697621628.py:6: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  res = pearsonr(calculated_values, annotated_values)
